In [ ]:
# Import necessary libraries
!pip install umap-learn

In [2]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd
from utils import *

In [8]:
# change the working directory to the location of the script
from pathlib import Path

data_file = Path(r"D:\gperez\data\ad_mci_hc\transcripts_matched_group.csv")
df = pd.read_csv(data_file, sep=",", encoding="utf-16")
df["text"] = df["text"].apply(lambda x: x.replace("\n", " "))
df["text"] = df["text"].apply(lambda x: x.replace("\r", " "))

In [10]:
segments = load_video_information(df,"text")

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
df.loc[0,"text"]

In [ ]:
# Load the model
model = SentenceTransformer('PlanTL-GOB-ES/roberta-base-bne')

In [ ]:
# Compute embeddings
for segment in segments:
    segment.compute_sentence_embeddings(model)

In [ ]:
# Flatten all sentence embeddings across all segments
all_embeddings = []
all_sentences = []

for segment in segments:
    for sent_emb in segment.sentence_embeddings:
        all_embeddings.append(sent_emb.embedding)
        all_sentences.append(sent_emb.sentence)  # Optional: for labeling or inspection

In [ ]:
# Reduce to 2D using UMAP
umap_model = UMAP(n_components=2, random_state=42)
embedding_2d = umap_model.fit_transform(np.array(all_embeddings))


In [ ]:
# Create a DataFrame for Plotly
df = pd.DataFrame({
    'x': embedding_2d[:, 0],
    'y': embedding_2d[:, 1],
    'sentence': all_sentences
})

# Create interactive scatter plot
fig = px.scatter(df, x='x', y='y', hover_data=['sentence'], title="UMAP of Sentence Embeddings")
fig.show()
